In [1]:
# Import packages
import numpy as np
import pandas as pd
import os
import datetime as dt

In [2]:
# Set directories
main_dir = os.getcwd() 
data_dir = main_dir + '/raw_data'

# Set files to import
which_year = '2018'
if which_year == '2018':
    filenames = ['indego_trips_2018_q1', 'indego_trips_2018_q2',
                 'indego_trips_2018_q3']
elif which_year == '2017': 
    filenames = ['indego_trips_2017_q1', 'indego_trips_2017_q2',
             'indego_trips_2017_q3', 'indego_trips_2017_q4']
elif which_year == '2016': 
    filenames = ['indego_trips_2016_q1', 'indego_trips_2016_q2',
             'indego_trips_2016_q3', 'indego_trips_2016_q4']
column_names = ['trip_id', 'duration', 'start_time', 'end_time','start_station','start_lat','start_lon', 
                'end_station','end_lat','end_lon', 'bike_id', 'plan_duration', 'trip_route_category', 'passholder_type']
file_format = '.csv'

In [3]:
# Concatinate all quarters for a given year into a single data frame
df = pd.DataFrame(columns = column_names)
for x in range(len(filenames)): 
    current_file = os.path.join(data_dir + '/' + filenames[x] + file_format)
    temp = pd.read_csv(current_file)
    df = df.append(temp)

In [4]:
# Clean up based on duration
# 1) Drop all trips for which there is no duration information
df = df[pd.notnull(df['duration'])]
# 2) Identify potentially anomalous trips (based on length) 
maxDuration = df["duration"].mean() + df["duration"].std()*2
minDuration  = 1 #one minute
numDroppedBottom = 100*(len(df[df['duration']<=minDuration]))/(len(df['duration']))
numDroppedTop = 100*(len(df[df['duration']>=maxDuration]))/(len(df['duration']))
# 3) Drop outliers (and print proportions)
df = df[(df.duration < maxDuration) & (df.duration > minDuration)]
print("Percent of trips that are too short %0.1f." %numDroppedBottom)
print("Percent of trips that are too long %0.1f." %numDroppedTop)

Percent of trips that are too short 1.8.
Percent of trips that are too long 1.4.


In [5]:
# Clean up based on start station missing data
# 1) Drop all trips for which there is no duration information
df = df[pd.notnull(df['start_station'])]
# Drop all trips for which we don't have the station number. 
# Identify unknown stations and drop them
df['start_station_id'] = df['start_station'].astype(int)
df = df[(df.start_station_id > 3000) & (df.start_station_id < 4000)]

In [6]:
# extract date and hour from the start date to end date range
df['start_time_date'] = pd.to_datetime(df['start_time']).dt.to_period('D')
df['start_time_hour'] = pd.DatetimeIndex(df['start_time']).hour
#df['start_time'] = df['start_time'].dt.round("H")
first_day = df['start_time_date'].min().to_timestamp()
last_day = df['start_time_date'].max().to_timestamp()
timeseries =  pd.date_range(first_day, last_day, freq='H')

In [7]:
# reformat the starttime, so that its rounder per hour
df['start_time'] = pd.to_datetime(df['start_time'])
df['start_time'] = df['start_time'].dt.round("H")

In [8]:
# format a timeseries dataframe so we can join it with the trips
timeseries_columns = ['start_time_date', 'start_time_hour']
timeseries_df = pd.DataFrame(index=timeseries, columns=timeseries_columns).reset_index()
timeseries_columns.insert(0, 'timestamp')
timeseries_df.columns = timeseries_columns
timeseries_df['start_time_date'] = pd.to_datetime(timeseries_df['timestamp']).dt.to_period('D')
timeseries_df['start_time_hour'] = pd.DatetimeIndex(timeseries_df['timestamp']).hour
# drop timestamp, we don't need it anymore
timeseries_df['ts_timestamp'] = timeseries_df['timestamp']
timeseries_df = timeseries_df.drop(columns = 'timestamp')

In [9]:
timeseries_df

,start_time_date,start_time_hour,ts_timestamp
0,2018-01-01,0,2018-01-01 00:00:00
1,2018-01-01,1,2018-01-01 01:00:00
2,2018-01-01,2,2018-01-01 02:00:00
3,2018-01-01,3,2018-01-01 03:00:00
4,2018-01-01,4,2018-01-01 04:00:00
5,2018-01-01,5,2018-01-01 05:00:00
6,2018-01-01,6,2018-01-01 06:00:00
7,2018-01-01,7,2018-01-01 07:00:00
8,2018-01-01,8,2018-01-01 08:00:00
9,2018-01-01,9,2018-01-01 09:00:00


In [10]:
# Identify most valuable stations
station_df = df.groupby(['start_station'])['trip_id'].count() #identify unique trips and count them
how_many = 10
most_valuable_stations = station_df.sort_values(ascending = False).head(how_many).index.tolist()
print(how_many,"most valuable stations in rank-order are: ", most_valuable_stations)

10 most valuable stations in rank-order are:  [3010, 3021, 3054, 3023, 3045, 3057, 3020, 3032, 3108, 3012]


In [11]:
# make a new dataframe for the most valuable station
temp_station_3010 = df[df['start_station_id'] == 3010].reset_index()
temp_station_3010 = temp_station_3010.drop(columns = 'index')

In [12]:
# group by the number of mostr frequent trips
grouped_station_3010 = pd.DataFrame({'count':temp_station_3010.groupby(['start_time']).size()}).reset_index()

In [13]:
grouped_station_3010 = grouped_station_3010.set_index('start_time')
grouped_station_3010 = grouped_station_3010.reindex(timeseries, fill_value = 0)

In [14]:
# add date and time separately as columns
# make a column out of an index
grouped_station_3010['timestamp'] = grouped_station_3010.index
grouped_station_3010['start_time_date'] = pd.to_datetime(grouped_station_3010['timestamp']).dt.to_period('D')
grouped_station_3010['start_time_hour'] = pd.DatetimeIndex(grouped_station_3010['timestamp']).hour
grouped_station_3010 = grouped_station_3010.reset_index()
grouped_station_3010 = grouped_station_3010.drop(columns = 'index')

In [15]:
# save into dataframe
filename = 'Station3010-' + which_year + file_format
grouped_station_3010.to_csv(filename)

In [18]:
grouped_station_3010

,count,timestamp,start_time_date,start_time_hour
0,0,2018-01-01 00:00:00,2018-01-01,0
1,0,2018-01-01 01:00:00,2018-01-01,1
2,0,2018-01-01 02:00:00,2018-01-01,2
3,0,2018-01-01 03:00:00,2018-01-01,3
4,0,2018-01-01 04:00:00,2018-01-01,4
5,0,2018-01-01 05:00:00,2018-01-01,5
6,0,2018-01-01 06:00:00,2018-01-01,6
7,0,2018-01-01 07:00:00,2018-01-01,7
8,0,2018-01-01 08:00:00,2018-01-01,8
9,0,2018-01-01 09:00:00,2018-01-01,9
